# Estimativa de custo e tempo de transporte áereo por distância

In [7]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd

In [8]:
id_aeroporto = pd.read_excel('data/excel/xlsx/id_aeroporto_v2.xlsx', engine='openpyxl')

In [9]:
id_aeroporto.head(1)

,SIGLA,ID_AEROPORTO,UF,LATITUDE,LONGITUDE,CLASSIF,PASSAGEIROS_ULTIMO_ANO,ORIGENS,DESTINOS,COD_IATA
0,SBCF,SBCF - CONFINS,MG,-19.624444,-43.971944,Classe IV,12435946,"['SBAX - ARAXÁ', 'SBGV - GOVERNADOR VALADARES'...","['SBAX - ARAXÁ', 'SBJD - JUNDIAÍ', 'SBVG - VAR...",CNF


In [10]:
# Filtrar para que apenas as linhas cuja coluna 'COD_IATA' seja diferente de NaN sejam mantidas
id_aeroporto_filtrado = id_aeroporto[id_aeroporto['COD_IATA'].notna()]

In [11]:
def buscar_origens(id_aeroporto_filtrado):
    # Configurar as opções do Chrome
    chrome_options = Options()
    chrome_options.add_argument("--start-maximized")  # Iniciar maximizado
    # Inicializar o driver do Chrome
    service = ChromeService()
    driver = webdriver.Chrome(service=service, options=chrome_options)

    # URL do Google Flights
    url = "https://www.google.com/travel/flights?tcfs&ved=2ahUKEwjTz-ijgK2JAxXVN7kGHSDtEJkQ0I8EegQIAxAJ&ictx=2"
    driver.get(url)

    # Esperar que o botão de lista suspensa esteja presente e clicar para abrir a lista de tipo de viagem
    WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, "div.VfPpkd-aPP78e"))
    ).click()

    # Selecionar a opção "Só ida"
    WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//li[contains(., 'Só ida')]"))
    ).click()

    # DataFrame para armazenar os resultados
    od_aerea_origens = pd.DataFrame(columns=['cod_iata', 'origem'])

    # Iterar pelos COD_IATA
    for cod_iata in id_aeroporto_filtrado['COD_IATA']:
        try:
            # Esperar e selecionar o campo de origem
            origem_input = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, "input[aria-label='De onde?']"))
            )

            # Verificar se há algum valor preenchido e limpar o campo
            if origem_input.get_attribute('value'):
                origem_input.clear()  # Limpar o valor atual do campo
                origem_input.send_keys(Keys.BACKSPACE * len(origem_input.get_attribute('value')))
            
            # Clicar no campo para garantir que está em foco
            origem_input.click()

            # Inserir o COD_IATA
            origem_input.send_keys(cod_iata)

            # Esperar pela lista de sugestões de aeroportos
            suggestions = WebDriverWait(driver, 10).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.CwL3Ec"))
            )

            # Procurar por um item na lista que corresponda exatamente ao COD_IATA
            found = False
            for suggestion in suggestions:
                iata_code_element = suggestion.find_element(By.CSS_SELECTOR, "div.P1pPOe")
                if iata_code_element.text.strip() == cod_iata:
                    nome_aeroporto = suggestion.find_element(By.CSS_SELECTOR, "div.zsRT0d").text
                    found = True
                    break

            # Se uma correspondência exata foi encontrada, adicionar ao DataFrame
            if found:
                novo_registro = pd.DataFrame({'cod_iata': [cod_iata], 'origem': [nome_aeroporto]})
                od_aerea_origens = pd.concat([od_aerea_origens, novo_registro], ignore_index=True)
                print(f"COD_IATA: {cod_iata}, Origem: {nome_aeroporto}")
            else:
                print(f"Nenhuma correspondência exata encontrada para {cod_iata}")

            # Pressionar ESC para fechar a lista suspensa
            origem_input.send_keys(Keys.ESCAPE)

        except Exception as e:
            print(f"Erro ao buscar origem para {cod_iata}: {e}")

    # Fechar o navegador após a iteração
    driver.quit()

    return od_aerea_origens

In [12]:
# Chame a função passando o DataFrame filtrado
od_aerea_origens = buscar_origens(id_aeroporto_filtrado)

Erro ao buscar origem para CNF: Message: 

Erro ao buscar origem para GIG: Message: element click intercepted: Element <input type="text" value="Florianópolis" jsname="yrriRe" jsaction="focus:h06R8; blur:zjh6rb" class="II2One j0Ppje zmMKJ LbIaRd VfPpkd-ksKsZd-mWPk3d-OWXEXe-AHe6Kc-XpnDCe" autocomplete="off" role="combobox" aria-autocomplete="inline" aria-haspopup="true" aria-expanded="false" aria-label="De onde?" jslog="8582;ved:2ahUKEwjPi6GezLGJAxWgBbkGHT3gBYkQhkN6BAgDEAw;track:click"> is not clickable at point (612, 472). Other element would receive the click: <input type="text" value="" jsname="yrriRe" jsaction="focus:h06R8; blur:zjh6rb" class="II2One j0Ppje zmMKJ LbIaRd" autocomplete="off" role="combobox" aria-autocomplete="both" aria-haspopup="true" aria-expanded="true" aria-label="De onde? " aria-describedby="i24" autofocus="" aria-owns="h0T7hb-7" tabindex="0">
  (Session info: chrome=130.0.6723.70)
Stacktrace:
	GetHandleVerifier [0x00007FF7732F3AB5+28005]
	(No symbol) [0x00007FF7

Exception ignored in: <function Service.__del__ at 0x000002C321292CA0>
Traceback (most recent call last):
  File "C:\Users\ecpro_ygnml4q\AppData\Roaming\Python\Python311\site-packages\selenium\webdriver\common\service.py", line 192, in __del__
    self.stop()
  File "C:\Users\ecpro_ygnml4q\AppData\Roaming\Python\Python311\site-packages\selenium\webdriver\common\service.py", line 149, in stop
    self.send_remote_shutdown_command()
  File "C:\Users\ecpro_ygnml4q\AppData\Roaming\Python\Python311\site-packages\selenium\webdriver\common\service.py", line 129, in send_remote_shutdown_command
    request.urlopen(f"{self.service_url}/shutdown")
  File "c:\ProgramData\anaconda3\Lib\urllib\request.py", line 216, in urlopen
    return opener.open(url, data, timeout)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\ProgramData\anaconda3\Lib\urllib\request.py", line 519, in open
    response = self._open(req, data)
               ^^^^^^^^^^^^^^^^^^^^^
  File "c:\ProgramData\anaconda3\Lib\urll